# ImageSorter

This notebook lets you sort a folder of images by perceptual similarity. Why would you want to do this? 
* Sort your StyleGAN seed images for smoother interpolation animations
* Sort generative image variations then use FiLM for cool animations
* Sort generative image variations then make into animated GIF
* Sort images for a collection and look for dupes or ones too similar
* Sort art work in a pleasing progression for a metaverse show
* Sort things for fun because you're bored 

<br>

Limitations:
* If you have a ton of images you may run out of RAM or finding the optimal order may take a long time.

<br>

If you're looking for more Ai art tools check out my [Ai generative art tools list](https://pharmapsychotic.com/tools.html).


In [ ]:
#@title Check GPU
!nvidia-smi -L

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title Setup
!pip install lpips==0.1.4 -q
!pip install python-tsp==0.4.1 -q

import lpips
import numpy as np
import os
import PIL.Image
import shutil
import torch
import torchvision.transforms.functional as TF
from tqdm import tqdm
from python_tsp.heuristics import solve_tsp_simulated_annealing

def load_image(path, dims):
    image = PIL.Image.open(path).convert("RGB")
    image = image.resize((dims, dims), PIL.Image.BILINEAR)
    return TF.to_tensor(image).to(device).mul(2).sub(1)

In [ ]:
#@title Let's sort some images!
images_folder = "/content/gdrive/MyDrive/AI/images" #@param {type:"string"}
sorted_folder = "/content/gdrive/MyDrive/AI/images/sorted" #@param {type:"string"}
perceptual_model = 'alex' #@param ['alex', 'squeeze', 'vgg']  
max_sort_minutes = 5 #@param {type: "integer"}

device = torch.device(f'cuda:0')
lpips_model = lpips.LPIPS(net=perceptual_model).to(device)

files = [file for file in os.listdir(images_folder) if os.path.splitext(file)[1] in ['.png', '.jpg', '.jpeg']]
if not len(files):
    raise Exception(f"No image files found in {images_folder}")

gpu_images = []
for file in tqdm(files, desc="Loading images"):
    gpu_images.append(load_image(os.path.join(images_folder, file), 128))

indexes = [i for i in range(len(gpu_images))]
distances = np.zeros((len(indexes), len(indexes)))
for a in tqdm(range(len(indexes)), desc="Computing distances"):
    for b in range(a+1, len(indexes)):
        diff = lpips_model(gpu_images[a], gpu_images[b])
        distances[a][b] = diff
        distances[b][a] = diff

print(f"Solving for optimal order using up to {max_sort_minutes} minutes...")
ordering, total_distance = solve_tsp_simulated_annealing(distances, max_processing_time=max_sort_minutes*60)

print(f"Saving in sorted order to {sorted_folder}...")
if not os.path.exists(sorted_folder):
    os.makedirs(sorted_folder)
for i in range(len(files)):
    file = files[ordering[i]] 
    base, ext = os.path.splitext(file)
    dest = os.path.join(sorted_folder, f"{i:04d}_{base}.{ext}")
    if os.path.exists(dest):
        os.remove(dest)
    shutil.copyfile(os.path.join(images_folder, file), dest)

print("Sorted order:")
print([os.path.splitext(files[ordering[i]])[0] for i in range(len(ordering))])
